In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!git clone https://github.com/rwightman/posenet-pytorch.git

Cloning into 'posenet-pytorch'...
remote: Enumerating objects: 115, done.
remote: Total 115 (delta 0), reused 0 (delta 0), pack-reused 115
Receiving objects: 100% (115/115), 34.73 KiB | 1.51 MiB/s, done.
Resolving deltas: 100% (62/62), done.


Необходимо предварительно загрузить архив `test.zip` в файловую систему (я использую Colab, поэтому пути далее ведут в `/content`)

In [ ]:
!unzip test.zip

Archive:  test.zip
   creating: test/
  inflating: test/oz_frame479.jpg    
  inflating: test/am3_6_violation_frame059.jpg  
  inflating: test/oz7_frame063.jpg   
  inflating: test/am3_9_frame001.jpg  
  inflating: test/oz7_frame512.jpg   
  inflating: test/oz_frame042.jpg    
  inflating: test/oz_violation_frame542.jpg  
  inflating: test/oz_frame254.jpg    
  inflating: test/oz_frame532.jpg    
  inflating: test/am3_6_frame042.jpg  
  inflating: test/am3_1_frame043.jpg  
  inflating: test/am3_5_frame117.jpg  
  inflating: test/oz7_frame465.jpg   
  inflating: test/am3_4_frame060.jpg  
  inflating: test/oz_frame127.jpg    
  inflating: test/oz7_frame064.jpg   
  inflating: test/am3_6_frame085.jpg  
  inflating: test/am3_6_frame052.jpg  
  inflating: test/oz7_frame514.jpg   
  inflating: test/am3_5_frame072.jpg  
  inflating: test/oz_frame481.jpg    
  inflating: test/am3_5_frame066.jpg  
  inflating: test/oz7_violation_frame489.jpg  
  inflating: test/oz_frame536.jpg    
  inflating: 

In [ ]:
%%writefile posenet-pytorch/image_demo.py
import cv2
import time
import argparse
import os
import torch

import posenet


parser = argparse.ArgumentParser()
parser.add_argument('--model', type=int, default=101)
parser.add_argument('--scale_factor', type=float, default=1.0)
parser.add_argument('--notxt', action='store_true')
parser.add_argument('--image_dir', type=str, default='./images')
parser.add_argument('--output_dir', type=str, default='./output')
args = parser.parse_args()


def main():
    model = posenet.load_model(args.model)
    #model = model.cuda()
    output_stride = model.output_stride

    if args.output_dir:
        if not os.path.exists(args.output_dir):
            os.makedirs(args.output_dir)

    filenames = [
        f.path for f in os.scandir(args.image_dir) if f.is_file() and f.path.endswith(('.png', '.jpg'))]

    start = time.time()
    for f in filenames:
        input_image, draw_image, output_scale = posenet.read_imgfile(
            f, scale_factor=args.scale_factor, output_stride=output_stride)

        with torch.no_grad():
            input_image = torch.Tensor(input_image)#.cuda()

            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = model(input_image)

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
                heatmaps_result.squeeze(0),
                offsets_result.squeeze(0),
                displacement_fwd_result.squeeze(0),
                displacement_bwd_result.squeeze(0),
                output_stride=output_stride,
                max_pose_detections=100,
                min_pose_score=0.15)

        keypoint_coords *= output_scale

        if args.output_dir:
            draw_image = posenet.draw_skel_and_kp(
                draw_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.25, min_part_score=0.25)

            cv2.imwrite(os.path.join(args.output_dir, os.path.relpath(f, args.image_dir)), draw_image)

        if not args.notxt:
            print()
            print("Results for image: %s" % f)
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                print('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))

    print('Average FPS:', len(filenames) / (time.time() - start))


if __name__ == "__main__":
    main()

Overwriting posenet-pytorch/image_demo.py


In [ ]:
import json

In [ ]:
!mkdir /content/boxes

In [ ]:
import cv2
import os
coco_data = json.load(open('subm.json'))
images = {}
for image in coco_data['images']:
    images[image['id']] = image

cadrs = []
offset = 40

for annotation in coco_data['annotations']:
    image = images[annotation['image_id']]
    source_path = 'test/' + image['file_name']
    #source_path = os.system(f'ls -1 test | grep {source_path}*')
    box_path = 'boxes/' + ''.join(image['file_name'].split('.')[:-1]) + '_' + str(annotation['id']) + '.jpg'
    bbox = annotation['bbox']
    img = cv2.imread(source_path)
    print(source_path)
    crop = img[max(0, bbox[1] - offset):min(bbox[1] + bbox[3] + offset, 1080), max(bbox[0] - offset, 0):min(bbox[0] + bbox[2] + offset, 1920)]
    print(crop.shape)
    if crop.shape[0] == 0 or crop.shape[1] == 0:
        continue
    cv2.imwrite(box_path, crop)
    cadrs.append({'source_path': source_path, 'bbox': bbox, 'box_path': box_path})

test/oz_violation_frame544.jpg
(259, 140, 3)
test/oz_violation_frame544.jpg
(216, 147, 3)
test/oz_violation_frame542.jpg
(258, 139, 3)
test/oz_violation_frame522.jpg
(289, 161, 3)
test/oz_violation_frame522.jpg
(317, 152, 3)
test/oz_violation_frame453.jpg
(350, 189, 3)
test/oz_violation_frame453.jpg
(381, 208, 3)
test/oz_violation_frame451.jpg
(384, 178, 3)
test/oz_violation_frame451.jpg
(337, 185, 3)
test/oz_violation_frame101.jpg
(368, 247, 3)
test/oz_violation_frame101.jpg
(351, 159, 3)
test/oz_violation_frame100.jpg
(373, 237, 3)
test/oz_violation_frame100.jpg
(339, 167, 3)
test/oz_violation_frame099.jpg
(352, 251, 3)
test/oz_violation_frame099.jpg
(376, 195, 3)
test/oz_violation_frame006.jpg
(293, 171, 3)
test/oz_violation_frame006.jpg
(322, 165, 3)
test/oz_frame617.jpg
(437, 243, 3)
test/oz_frame617.jpg
(509, 240, 3)
test/oz_frame613.jpg
(441, 216, 3)
test/oz_frame613.jpg
(407, 201, 3)
test/oz_frame583.jpg
(447, 229, 3)
test/oz_frame583.jpg
(512, 245, 3)
test/oz_frame574.jpg
(389

In [ ]:

!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [ ]:
!python posenet-pytorch/image_demo.py --model 101 --image_dir boxes --output_dir output

Выходные данные были обрезаны до нескольких последних строк (5000).
Keypoint rightAnkle, score = 0.588280, coord = [436.38826519  78.04659824]
Pose #1, score = 0.208097
Keypoint nose, score = 0.341081, coord = [180.98949187  34.01848473]
Keypoint leftEye, score = 0.215051, coord = [175.1277309   33.22438412]
Keypoint rightEye, score = 0.400947, coord = [174.60089157  25.54771997]
Keypoint leftEar, score = 0.236067, coord = [172.564014   29.6872699]
Keypoint rightEar, score = 0.675851, coord = [177.88981785   8.82696572]
Keypoint leftShoulder, score = 0.260139, coord = [232.77907901  10.82970122]
Keypoint rightShoulder, score = 0.209830, coord = [216.95551432  -6.63912768]
Keypoint leftElbow, score = 0.062907, coord = [299.25368623  29.3154804 ]
Keypoint rightElbow, score = 0.047160, coord = [280.07014083  -7.29899133]
Keypoint leftWrist, score = 0.010404, coord = [303.9139113   22.43069478]
Keypoint rightWrist, score = 0.020582, coord = [310.08012346 -11.18836626]
Keypoint leftHip, sco

In [ ]:
!zip -r output.zip output

  adding: output/ (stored 0%)
  adding: output/oz_frame127_70.jpg (deflated 1%)
  adding: output/oz7_violation_frame501_93.jpg (deflated 1%)
  adding: output/am3_5_frame041_251.jpg (deflated 0%)
  adding: output/oz_frame550_31.jpg (deflated 1%)
  adding: output/oz_frame551_30.jpg (deflated 1%)
  adding: output/am3_6_frame070_220.jpg (deflated 1%)
  adding: output/oz7_frame514_110.jpg (deflated 1%)
  adding: output/oz_frame482_45.jpg (deflated 1%)
  adding: output/am3_6_frame042_226.jpg (deflated 1%)
  adding: output/oz7_frame022_154.jpg (deflated 2%)
  adding: output/oz_frame559_28.jpg (deflated 1%)
  adding: output/oz7_frame556_107.jpg (deflated 2%)
  adding: output/am3_9_frame021_175.jpg (deflated 0%)
  adding: output/oz_frame151_65.jpg (deflated 0%)
  adding: output/am3_1_frame043_267.jpg (deflated 1%)
  adding: output/am3_7_frame150_185.jpg (deflated 2%)
  adding: output/am3_6_frame085_216.jpg (deflated 1%)
  adding: output/oz_frame430_55.jpg (deflated 0%)
  adding: output/am3_9_fr

In [ ]:
from google.colab import files
files.download('output.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>